# Avalanche Danger Level Forecast: Preliminary Modeling

## Import Tools

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

# pandas
from pandas.plotting import scatter_matrix

## Import Data

In [2]:
avi = pd.read_csv('SnowWeatherCleanROS.csv')
avi.head()

,Unnamed: 0,avi_danger,avg_wind,temp_max_swing,temp_max_swing_from_avg,year,month,day,temp_max,temp_min,...,prevailing_wind_N_2,prevailing_wind_NE_2,prevailing_wind_NW_2,prevailing_wind_S_2,prevailing_wind_SE_2,prevailing_wind_SW_2,prevailing_wind_W_2,three_day_snow_2,five_day_snow_2,next_day_avi_danger
0,0,1.0,20.58,0.0,0.0,2010.0,12.0,20.0,15,5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.2,0.2,3.0
1,1,3.0,35.12,3.0,0.0,2010.0,12.0,21.0,18,10,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.3,0.3,2.0
2,2,2.0,33.78,-3.0,0.0,2010.0,12.0,22.0,15,7,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,2.5,3.0
3,3,3.0,31.32,0.0,0.0,2010.0,12.0,23.0,15,6,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.3,4.5,2.0
4,4,2.0,32.44,2.0,1.4,2010.0,12.0,24.0,17,9,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,8.3,8.6,2.0


## Dataset at a Glance

In [3]:
avi.describe()

,Unnamed: 0,avi_danger,avg_wind,temp_max_swing,temp_max_swing_from_avg,year,month,day,temp_max,temp_min,...,prevailing_wind_N_2,prevailing_wind_NE_2,prevailing_wind_NW_2,prevailing_wind_S_2,prevailing_wind_SE_2,prevailing_wind_SW_2,prevailing_wind_W_2,three_day_snow_2,five_day_snow_2,next_day_avi_danger
count,1876.000000,1876.000000,1876.000000,1876.000000,1876.000000,1876.00000,1876.000000,1876.000000,1876.000000,1876.000000,...,1876.000000,1876.000000,1876.000000,1876.000000,1876.000000,1876.000000,1876.000000,1876.000000,1876.000000,1876.000000
mean,937.500000,2.187100,42.177013,0.372068,0.629531,2015.27452,3.573028,16.812900,19.523454,3.563966,...,0.060768,0.015991,0.281983,0.055970,0.022921,0.066098,0.479744,4.609168,7.521002,2.500000
std,541.698871,0.890641,15.850886,10.967304,12.506157,2.95618,3.288363,8.399852,13.053530,13.952745,...,0.238967,0.125476,0.450085,0.229925,0.149692,0.248520,0.499723,5.106127,6.670821,1.118332
min,0.000000,1.000000,4.250000,-44.000000,-53.800000,2010.00000,1.000000,1.000000,-26.000000,-40.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,468.750000,1.000000,30.420000,-5.000000,-7.400000,2013.00000,2.000000,11.000000,11.000000,-6.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.900000,1.750000
50%,937.500000,2.000000,41.160000,2.000000,1.000000,2015.00000,3.000000,18.000000,19.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.900000,5.500000,2.500000
75%,1406.250000,3.000000,53.020000,7.000000,9.800000,2018.00000,4.000000,24.000000,28.000000,12.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,6.200000,10.800000,3.250000
max,1875.000000,4.000000,110.060000,39.000000,34.600000,2020.00000,12.000000,31.000000,57.000000,43.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,34.200000,46.000000,4.000000


In [4]:
avi.columns

Index(['Unnamed: 0', 'avi_danger', 'avg_wind', 'temp_max_swing',
       'temp_max_swing_from_avg', 'year', 'month', 'day', 'temp_max',
       'temp_min', 'water_equivalent', 'snow_fall', 'snow_depth_6am',
       'wind_speed_sum', 'sunshine_percent', 'west_wind_hours',
       'northwest_wind_hours', 'prevailing_wind_E', 'prevailing_wind_N',
       'prevailing_wind_NE', 'prevailing_wind_NW', 'prevailing_wind_S',
       'prevailing_wind_SE', 'prevailing_wind_SW', 'prevailing_wind_W',
       'three_day_snow', 'five_day_snow', 'avi_danger_1', 'avg_wind_1',
       'temp_max_swing_1', 'temp_max_swing_from_avg_1', 'temp_max_1',
       'temp_min_1', 'water_equivalent_1', 'snow_fall_1', 'snow_depth_6am_1',
       'wind_speed_sum_1', 'sunshine_percent_1', 'west_wind_hours_1',
       'northwest_wind_hours_1', 'prevailing_wind_E_1', 'prevailing_wind_N_1',
       'prevailing_wind_NE_1', 'prevailing_wind_NW_1', 'prevailing_wind_S_1',
       'prevailing_wind_SE_1', 'prevailing_wind_SW_1', 'prevailing_

In [5]:
avi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1876 entries, 0 to 1875
Data columns (total 74 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 1876 non-null   int64  
 1   avi_danger                 1876 non-null   float64
 2   avg_wind                   1876 non-null   float64
 3   temp_max_swing             1876 non-null   float64
 4   temp_max_swing_from_avg    1876 non-null   float64
 5   year                       1876 non-null   float64
 6   month                      1876 non-null   float64
 7   day                        1876 non-null   float64
 8   temp_max                   1876 non-null   int64  
 9   temp_min                   1876 non-null   int64  
 10  water_equivalent           1876 non-null   float64
 11  snow_fall                  1876 non-null   float64
 12  snow_depth_6am             1876 non-null   float64
 13  wind_speed_sum             1876 non-null   int64

## Filter Dataset

In [6]:
avi = avi.drop(['Unnamed: 0'], axis=1)

### Standardizing the Data

In [7]:
#standardizer 
def standardize(X_train, X_test):
    scaler = StandardScaler()
    # Fitting and transforming training data
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    # Tranforming testing data based on traning fit (prevent data leakage)
    X_test = scaler.transform(X_test)
    return X_train, X_test

## Splitting Dataset

In [8]:
x = avi.iloc[:, 0:avi.shape[1]-1]
y = avi.iloc[:, avi.shape[1]-1]

print(x.shape)
print(y.shape)

(1876, 72)
(1876,)


In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [10]:
# Scaling
x_train, x_test = standardize(x_train, x_test)

In [11]:
print(y)

0       3.0
1       2.0
2       3.0
3       2.0
4       2.0
       ... 
1871    4.0
1872    4.0
1873    4.0
1874    4.0
1875    4.0
Name: next_day_avi_danger, Length: 1876, dtype: float64


# Logistic Regression

In [12]:
penalty = ['l1', 'l2', 'elasticnet', 'none']
tol = [0.0001, 0.001, 0.01, 0.1]
C = [0.001, 0.01, 0.1, 1, 10, 100, 1000] 
solver = ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']
param_distributions = dict(penalty=penalty,
                           tol=tol,
                           C=C,
                           solver = solver)


lr = LogisticRegression(class_weight = 'balanced',max_iter = 1000)
lr_cv = RandomizedSearchCV(estimator=lr, param_distributions= param_distributions, n_iter=5, scoring='f1_weighted')
lr_cv.fit(x_train, y_train)
y_pred_lr = lr_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_lr))
print(cross_val_score(lr, x_train, y_train, cv=3))

/Users/matthewthompson/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/matthewthompson/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/matthewthompson/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/matthewthompson/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 453, in _check_solver
    raise ValueError(
ValueError: penalty='none' is not supported for the liblinear solver

  warnings.warn("Estimator fit failed. The score on this train-test"
/Users/matthewthompson/opt/anaconda3/lib/pyth

0.6140724946695096
[0.62899787 0.60127932 0.64392324]


In [13]:
print(confusion_matrix(y_test, y_pred_lr))

[[91 13  2  5]
 [25 59 25 12]
 [ 9 26 53 31]
 [ 8  8 17 85]]


In [14]:
print(classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

         1.0       0.68      0.82      0.75       111
         2.0       0.56      0.49      0.52       121
         3.0       0.55      0.45      0.49       119
         4.0       0.64      0.72      0.68       118

    accuracy                           0.61       469
   macro avg       0.61      0.62      0.61       469
weighted avg       0.60      0.61      0.61       469



# Support Vector Machine (SVM)

In [15]:
param_distributions = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  


svm = SVC()
svm_cv = RandomizedSearchCV(estimator=svm, param_distributions= param_distributions, n_iter=5, scoring='f1_weighted')
svm_cv.fit(x_train, y_train)
y_pred_svm = svm_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_svm))
print(cross_val_score(svm, x_train, y_train, cv=3))

0.7590618336886994
[0.69936034 0.65245203 0.70788913]


In [16]:
print(confusion_matrix(y_test, y_pred_svm))

[[ 89  19   3   0]
 [ 25  73  20   3]
 [  6  28  77   8]
 [  0   1   0 117]]


In [17]:
print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

         1.0       0.74      0.80      0.77       111
         2.0       0.60      0.60      0.60       121
         3.0       0.77      0.65      0.70       119
         4.0       0.91      0.99      0.95       118

    accuracy                           0.76       469
   macro avg       0.76      0.76      0.76       469
weighted avg       0.76      0.76      0.76       469



# K-Nearest Neighbors

In [18]:
#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
#Convert to dictionary
param_distributions = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

knn = KNeighborsClassifier()
knn_cv = RandomizedSearchCV(estimator=knn, param_distributions= param_distributions, n_iter=5, scoring='f1_weighted')
knn_cv.fit(x_train, y_train)
y_pred_knn = knn_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_knn))
print(cross_val_score(knn, x_train, y_train, cv=3))

0.5906183368869936
[0.58635394 0.57782516 0.61407249]


In [19]:
print(confusion_matrix(y_test, y_pred_knn))

[[79  9  7 16]
 [31 50 20 20]
 [16 18 62 23]
 [ 4 10 18 86]]


In [20]:
print(classification_report(y_test, y_pred_knn))

              precision    recall  f1-score   support

         1.0       0.61      0.71      0.66       111
         2.0       0.57      0.41      0.48       121
         3.0       0.58      0.52      0.55       119
         4.0       0.59      0.73      0.65       118

    accuracy                           0.59       469
   macro avg       0.59      0.59      0.58       469
weighted avg       0.59      0.59      0.58       469



# Gradient Boosting Classifier

In [21]:
param_distributions = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 
                       'n_estimators':[100,250,500,750,1000,1250,1500,1750],
                       'max_depth':[2,3,4,5,6,7] }

gbc = GradientBoostingClassifier()
gbc_cv = RandomizedSearchCV(estimator=gbc, param_distributions= param_distributions, n_iter=5, scoring='f1_weighted')
gbc.fit(x_train, y_train)
y_pred_gbc = gbc.predict(x_test)
print(accuracy_score(y_test, y_pred_gbc))
print(cross_val_score(gbc, x_train, y_train, cv=3))

0.7974413646055437
[0.75053305 0.7228145  0.74840085]


In [22]:
print(confusion_matrix(y_test, y_pred_gbc))

[[ 91  15   4   1]
 [ 15  80  24   2]
 [  7  21  86   5]
 [  0   1   0 117]]


In [24]:
print(classification_report(y_test, y_pred_gbc))

              precision    recall  f1-score   support

         1.0       0.81      0.82      0.81       111
         2.0       0.68      0.66      0.67       121
         3.0       0.75      0.72      0.74       119
         4.0       0.94      0.99      0.96       118

    accuracy                           0.80       469
   macro avg       0.79      0.80      0.80       469
weighted avg       0.79      0.80      0.80       469

